In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
train_data_dir='/content/gdrive/My Drive/lung_colon_image_set/colon_image_sets'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=45,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(150, 150),
    batch_size=45,
    class_mode='binary',
    subset='validation') # set as validation data



Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
# example of tending the vgg16 model
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
# load model without classifier layers
model =  VGG16(include_top=False, input_shape=(150, 150, 3))
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(2, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer= opt, loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# simple early stopping
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=50)
mc = ModelCheckpoint('/content/gdrive/My Drive/model/best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
batch_size=45
nb_epochs=1500
history=model.fit(
    train_generator,
    batch_size=45,
    validation_data = validation_generator,

    epochs = nb_epochs,callbacks=[es])

Epoch 1/1500
178/178 [==============================] - 158s 886ms/step - loss: 3.2437 - accuracy: 0.4955 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/1500
178/178 [==============================] - 157s 885ms/step - loss: 0.6933 - accuracy: 0.4941 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/1500
178/178 [==============================] - 157s 884ms/step - loss: 0.6933 - accuracy: 0.4967 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/1500
178/178 [==============================] - 158s 886ms/step - loss: 0.6932 - accuracy: 0.4913 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/1500
178/178 [==============================] - 158s 890ms/step - loss: 0.6933 - accuracy: 0.4946 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/1500
178/178 [==============================] - 158s 888ms/step - loss: 0.6932 - accuracy: 0.4950 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/1500
178/178 [==============================] - 157s 883ms/step - loss: 0.6932 - accuracy: 0.4934 - val_

In [ ]:
test_directory='/content/gdrive/My Drive/CAT_DOGS/2Class/testing_data'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_directory, # same directory as training data
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')


Found 400 images belonging to 2 classes.


In [ ]:
score = model.evaluate(test_generator)


13/13 [==============================] - 185s 14s/step - loss: 0.6932 - accuracy: 0.5000


In [ ]:
print(score)


[0.6931523680686951, 0.5]


In [ ]:
#Confution Matrix and Classification Report
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
num_of_test_samples=400
Y_pred = model.predict(test_generator, num_of_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['adenocarcinomas', 'benign']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[200   0]
 [200   0]]
Classification Report
                 precision    recall  f1-score   support

adenocarcinomas       0.50      1.00      0.67       200
         benign       0.00      0.00      0.00       200

       accuracy                           0.50       400
      macro avg       0.25      0.50      0.33       400
   weighted avg       0.25      0.50      0.33       400



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
